# Environment setting
環境設定

In [1]:
import os
import requests
import sys
from pathlib import Path
from pprint import pprint
from typing import Optional

import pandas as pd


# determine branch, default is main
branch: str = "main"

# determine subfolder, default is None (petsard/demo/)
subfolder: Optional[str] = "use-cases/preproc"


# Check if running in Google Colab, if so, download the utils.py file from GitHub
is_colab: bool = "COLAB_GPU" in os.environ
if is_colab:
    utils_url = (
        f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    )
    response = requests.get(utils_url)

    if response.status_code == 200:
        with open("utils.py", "w") as f:
            f.write(response.text)

        Path("__init__.py").touch()
    else:
        raise RuntimeError(
            f"Failed to download utils.py. Status code: {response.status_code}"
        )


# If not colab, and also contains subfolder, add the correct path of util.py
else:
    if subfolder:
        sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

In [ ]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        "adult-income",
    ],
    subfolder=subfolder,
)

In [3]:
from petsard import Executor

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

## Handling Missing Values
資料遺失值處理

In [4]:
yaml_file_case: str = "handling-missing-values.yaml"

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
    subfolder=subfolder,
)

Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/adult-income.csv'
    na_values: '?' # every '?' in the dataset will be considered as missing value
Preprocessor:
  missing-only:
    # only execute the missing values handler and encoding by their default,
    #   the rest of the preprocessing steps will be skipped
    # keep encoding due to we have categorical features
    sequence:
      - 'missing'
      - 'encoder'
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Evaluator:
  demo-quality:
    method: 'sdmetrics-qualityreport'
Reporter:
  output:
    method: 'save_data'
    source: 'Synthesizer'
  save_report_global:
    method: 'save_report'
    granularity: 'global'
...


### Execution and Result
執行與結果

In [5]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 78.45it/s]|
Column Shapes Score: 89.69%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:00<00:00, 175.66it/s]|
Column Pair Trends Score: 79.96%

Overall Score (Average): 84.83%

Now is petsard_Loader[data]_Preprocessor[missing-only]_Synthesizer[demo] save to csv...
Now is petsard[Report]_[global] save to csv...


In [6]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[missing-only]_Synthesizer[demo]_Postprocessor[demo]_Evaluator[demo-quality]_Reporter[save_report_global]"
]["[global]"]

,full_expt_name,Loader,Preprocessor,Synthesizer,Postprocessor,Evaluator,demo-quality_Score,demo-quality_Column Shapes,demo-quality_Column Pair Trends
result,Loader[data]_Preprocessor[missing-only]_Synthe...,data,missing-only,demo,demo,[global],0.85,NaN,NaN
